In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import fractional_matrix_power  # 计算矩阵的分数次幂
from scipy.sparse.linalg import eigs  # 计算矩阵的特征值和特征向量

In [7]:
class SIR:
    def __init__(self, H, K, estdim=0, Cn=1):
        self.H = H
        self.K = K
        self.estdim = estdim
        self.Cn = Cn

    def fit(self, X, Y):
        self.X = X
        self.Y = Y
        self.mean_x = np.mean(X, axis=0)  # 计算样本均值
        self.sigma_x = np.cov(X, rowvar=False)  # 计算样本协方差矩阵
        self.Z = np.matmul(X - np.tile(self.mean_x, (X.shape[0], 1)),
                           fractional_matrix_power(self.sigma_x, -0.5))  # 标准化后的数据阵
        n, p = self.Z.shape
        if self.Y.ndim == 1:  # 判断响应变量Y的维度
            self.Y = self.Y.reshape(-1, 1)
        ny, py = self.Y.shape
        W = np.ones((n, 1)) / n
        nw, pw = W.shape
        # 输入数据异常判断
        if n != ny:
            raise ValueError('X and Y must have the same number of samples')
        elif p == 1:
            raise ValueError('X must have at least 2 dimensions')
        elif py != 1:
            raise ValueError('Y must have only 1 dimension')
        # 将y分为H片，c为每个片的样本数
        c = np.ones((1, self.H)) * (n // self.H) + np.hstack(
            [np.ones(
                (1, n % self.H)), np.zeros((1, self.H - n % self.H))])
        cumc = np.cumsum(c)  # 计算切片累计和
        # 参照Y的取值从小到大进行排序
        temp = np.hstack((self.Z, self.Y, W))
        temp = temp[np.argsort(temp[:, p])]
        # 提取排序后的z,y,w
        z, y, w = temp[:, :p], temp[:, p:p + 1], temp[:, p + 1]
        muh = np.zeros((self.H, p))
        wh = np.zeros((self.H, 1))  # 每个切片的权重
        k = 0  # 初始化切片编号
        for i in range(n):
            if i >= cumc[k]:  # 如果超过了切片的边界，则换下一个切片
                k += 1
            muh[k, :] = muh[k, :] + z[i, :]  # 计算切片内自变量之和
            wh[k] = wh[k] + w[i]  # 计算每个切片包含Yi的概率
        # 计算每个切片的样本均值,将其作为切片内自变量的取值
        muh = muh / (np.tile(wh, (1, p)) * n)
        # 加权主成分分析
        self.M = np.zeros((p, p))  # 初始化切片 加权协方差矩阵
        for i in range(self.H):
            self.M = self.M + wh[i] * muh[i, :].reshape(-1, 1) * muh[i, :]
        if self.estdim == 0: # 一般情况
            self.D, self.V = eigs(A=self.M, k=self.K, which='LM')
        else:
            """ # 稀疏矩阵情况，待修正
            [V D] = np.linalg,eig(full(M))
            lambda = np.sort(abs(diag(D)),'descend')
            L = np.log(lambda+1) - lambda
            G = np.zeros((p,1))
            if Cn == 1
                Cn = n^(1/2)
            elif Cn == 2
            Cn = n^(1/3)
            elif Cn == 3:
                Cn = 0.5 * n^0.25
            for k in range(p):
                G(k) = n / 2 * sum(L(1:k)) / sum(L) - Cn * k * (k+1) / p
            maxG, K = np. max(G)
            V, D = eigs(M,K,'lm')
            """
            pass
        return self.V, self.K, self.M, self.D
    def transform(self):
        hatbeta = np.matmul(fractional_matrix_power(self.sigma_x, -0.5),self.V)
        return hatbeta

In [10]:
X = pd.read_csv('X.csv',header=None).values
Y = pd.read_csv('Y.csv',header=None).values
model = SIR(H=6, K=4, estdim=0, Cn=1)
V, K, M, D = model.fit(X,Y)
hatbeta = model.transform()

# Test

In [ ]:
from scipy.linalg import fractional_matrix_power  # 计算矩阵的分数次幂
X = pd.read_csv('X.csv',header=None).values
Y = pd.read_csv('Y.csv',header=None).values
mean_x =  np.mean(X, axis=0)  # 计算X的均值
sigma_x = np.cov(X, rowvar=False)  # 计算X的协方差矩阵
Z = np.matmul(X - np.tile(mean_x, (X.shape[0], 1)),
                           fractional_matrix_power(sigma_x, -0.5))  # 标准化后的数据阵

In [501]:
n, p = Z.shape
ny, py = Y.reshape(-1,1).shape
estdim = 0
Cn = 1
H = 6
K=5
W = np.ones((n,1))/n
nw,pw = W.shape
c = np.ones((1,H)) * (n//H) + np.hstack([np.ones((1,n%H)), np.zeros((1,H-n%H))])
cumc = np.cumsum(c)
temp = np.hstack((Z,Y, W))
temp = temp[np.argsort(temp[:,p])] # 按照Y进行排序
# 提取排序后的z,y,w
z,y,w = temp[:,:p],temp[:,p:p+1],temp[:,p+1]
muh = np.zeros((H,p)) # 初始化切片均值
wh = np.zeros((H,1)) # 初始化每个切片包含Yi的概率
k = 0 # 初始化切片编号
for i in range(n):
    if i >= cumc[k]: # 如果超过了切片的边界，则换下一个切片
        k += 1
    muh[k,:] = muh[k,:] + z[i,:] # 计算切片内自变量之和
    wh[k] = wh[k] + w[i] # 计算每个切片包含Yi的概率
# 计算每个切片的样本均值,将其作为切片内自变量的取值
muh = muh / (np.tile(wh, (1, p))*n) 
M = np.zeros((p,p)) # 初始化切片 加权协方差矩阵
for i in range(H):
    M = M + wh[i] * muh[i,:].reshape(-1,1) * muh[i,:]
D,V = eigs(A=M,k=K,which='LM')